In [ ]:
# %% [markdown]
# # Design of Experiments: Identifying Critical Factors in Chemical Reactions
# 
# This notebook demonstrates the complete workflow for screening experiments:
# 1. Generate experimental design
# 2. Analyze results to identify critical factors
# 3. Visualize factor importance
# 4. Statistical significance testing
sd
# %% [markdown]
# ## 1. Setup and Data Generation

# %%
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# %% [markdown]
# ## 2. Generate Screening Design (Plackett-Burman)

# %%
def generate_plackett_burman_design(factors):
    """
    Generate a Plackett-Burman screening design
    """
    n_factors = len(factors)
    
    # 8-run Plackett-Burman design matrix (for up to 7 factors)
    if n_factors <= 7:
        design_matrix = np.array([
            [1, 1, 1, -1, 1, -1, -1],
            [1, 1, -1, 1, -1, -1, 1],
            [1, -1, 1, -1, -1, 1, 1],
            [-1, 1, -1, -1, 1, 1, 1],
            [1, -1, -1, 1, 1, 1, -1],
            [-1, -1, 1, 1, 1, -1, 1],
            [-1, 1, 1, 1, -1, 1, -1],
            [-1, -1, -1, -1, -1, -1, -1]
        ])[:, :n_factors]
    else:
        # 12-run design for 8-11 factors
        design_matrix = np.array([
            [1, 1, -1, 1, 1, 1, -1, -1, -1, 1, -1],
            [1, -1, 1, 1, 1, -1, -1, -1, 1, -1, 1],
            [-1, 1, 1, 1, -1, -1, -1, 1, -1, 1, 1],
            [1, 1, 1, -1, -1, -1, 1, -1, 1, 1, -1],
            [1, 1, -1, -1, -1, 1, -1, 1, 1, -1, 1],
            [1, -1, -1, -1, 1, -1, 1, 1, -1, 1, 1],
            [-1, -1, -1, 1, -1, 1, 1, -1, 1, 1, 1],
            [-1, -1, 1, -1, 1, 1, -1, 1, 1, 1, -1],
            [-1, 1, -1, 1, 1, -1, 1, 1, 1, -1, -1],
            [1, -1, 1, 1, -1, 1, 1, 1, -1, -1, -1],
            [-1, 1, 1, -1, 1, 1, 1, -1, -1, -1, 1],
            [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
        ])[:, :n_factors]
    
    # Convert coded values to actual values
    design_df = pd.DataFrame(design_matrix, columns=[f['name'] for f in factors])
    
    for i, factor in enumerate(factors):
        design_df[factor['name']] = design_df[factor['name']].map({
            -1: factor['low'],
            1: factor['high']
        })
    
    return design_df, design_matrix

# %%
# Define factors for a chemical reaction
factors = [
    {'name': 'Temperature', 'low': 20, 'high': 80, 'unit': '°C'},
    {'name': 'pH', 'low': 4, 'high': 9, 'unit': ''},
    {'name': 'Catalyst_Conc', 'low': 0.5, 'high': 2.0, 'unit': 'g/L'},
    {'name': 'Pressure', 'low': 1, 'high': 5, 'unit': 'bar'},
    {'name': 'Reaction_Time', 'low': 30, 'high': 120, 'unit': 'min'},
    {'name': 'Stirring_Speed', 'low': 200, 'high': 600, 'unit': 'rpm'},
    {'name': 'Substrate_Conc', 'low': 0.1, 'high': 0.5, 'unit': 'M'}
]

# Generate design
design_df, coded_design = generate_plackett_burman_design(factors)
print("Experimental Design Matrix:")
print(design_df)
print(f"\nTotal number of runs: {len(design_df)}")

# %% [markdown]
# ## 3. Simulate Response Data
# 
# In practice, you would run actual experiments and measure the response.
# Here we simulate a response where Temperature, pH, and Catalyst are critical factors.

# %%
def simulate_response(design_matrix, factors, noise_level=0.1):
    """
    Simulate response with known factor effects
    Temperature and pH are most critical, with interaction
    """
    # True effects (unknown in real experiments)
    true_effects = {
        'Temperature': 15.0,      # Large positive effect
        'pH': -12.0,              # Large negative effect
        'Catalyst_Conc': 8.0,     # Medium positive effect
        'Pressure': 2.0,          # Small positive effect
        'Reaction_Time': -1.5,    # Small negative effect
        'Stirring_Speed': 0.5,    # Negligible effect
        'Substrate_Conc': 1.0     # Small positive effect
    }
    
    baseline = 65.0  # Baseline yield
    
    # Calculate response
    response = np.ones(len(design_matrix)) * baseline
    
    for i, factor in enumerate(factors):
        effect = true_effects[factor['name']]
        response += coded_design[:, i] * effect / 2  # Divide by 2 for half-effect
    
    # Add interaction between Temperature and pH (they amplify each other)
    temp_idx = [f['name'] for f in factors].index('Temperature')
    ph_idx = [f['name'] for f in factors].index('pH')
    interaction_effect = -5.0
    response += coded_design[:, temp_idx] * coded_design[:, ph_idx] * interaction_effect / 2
    
    # Add random noise
    noise = np.random.normal(0, noise_level * np.std(response), len(response))
    response += noise
    
    return response, true_effects

# Set random seed for reproducibility
np.random.seed(42)

# Simulate experimental results
yield_response, true_effects = simulate_response(design_df, factors)
design_df['Yield'] = yield_response

print("\nExperimental Results (simulated):")
print(design_df)

# %% [markdown]
# ## 4. Calculate Factor Effects

# %%
def calculate_effects(coded_design, response, factor_names):
    """
    Calculate main effects for each factor
    """
    n_runs = len(response)
    effects = []
    
    for i in range(coded_design.shape[1]):
        # Effect = average at high level - average at low level
        high_avg = np.mean(response[coded_design[:, i] == 1])
        low_avg = np.mean(response[coded_design[:, i] == -1])
        effect = high_avg - low_avg
        effects.append(effect)
    
    effects_df = pd.DataFrame({
        'Factor': factor_names,
        'Effect': effects,
        'Abs_Effect': np.abs(effects)
    })
    
    return effects_df.sort_values('Abs_Effect', ascending=False)

# Calculate effects
factor_names = [f['name'] for f in factors]
effects_df = calculate_effects(coded_design, yield_response, factor_names)

print("\nFactor Effects (sorted by importance):")
print(effects_df)

# %% [markdown]
# ## 5. Statistical Analysis with Linear Regression

# %%
# Fit linear regression model
X = coded_design
y = yield_response

model = LinearRegression()
model.fit(X, y)

# Get coefficients (half-effects)
coefficients = model.coef_
effects_regression = coefficients * 2  # Convert to full effects

# Calculate R² score
r2_score = model.score(X, y)
print(f"\nModel R² Score: {r2_score:.4f}")
print(f"Model explains {r2_score*100:.2f}% of variance in the response")

# Calculate standard errors and p-values
y_pred = model.predict(X)
residuals = y - y_pred
mse = np.sum(residuals**2) / (len(y) - len(coefficients) - 1)
X_with_intercept = np.column_stack([np.ones(len(X)), X])
cov_matrix = mse * np.linalg.inv(X_with_intercept.T @ X_with_intercept)
se = np.sqrt(np.diag(cov_matrix)[1:])  # Skip intercept

# Calculate t-statistics and p-values
t_stats = coefficients / se
df = len(y) - len(coefficients) - 1
p_values = 2 * (1 - stats.t.cdf(np.abs(t_stats), df))

# Create results dataframe
results_df = pd.DataFrame({
    'Factor': factor_names,
    'Effect': effects_regression,
    'Coefficient': coefficients,
    'Std_Error': se,
    't_statistic': t_stats,
    'p_value': p_values,
    'Significant': ['***' if p < 0.001 else '**' if p < 0.01 else '*' if p < 0.05 else 'ns' 
                    for p in p_values]
})
results_df = results_df.sort_values('Effect', key=abs, ascending=False)

print("\n" + "="*80)
print("STATISTICAL SIGNIFICANCE TESTING")
print("="*80)
print(results_df.to_string(index=False))
print("\nSignificance codes: *** p<0.001, ** p<0.01, * p<0.05, ns = not significant")

# %% [markdown]
# ## 6. Visualization: Pareto Chart of Effects

# %%
fig, ax = plt.subplots(figsize=(10, 6))

# Sort by absolute effect
sorted_results = results_df.sort_values('Effect', key=abs, ascending=True)

# Color bars based on significance
colors = ['red' if sig in ['***', '**', '*'] else 'lightgray' 
          for sig in sorted_results['Significant']]

# Create horizontal bar chart
bars = ax.barh(range(len(sorted_results)), sorted_results['Effect'], color=colors)

# Add significance markers
for i, (effect, sig) in enumerate(zip(sorted_results['Effect'], sorted_results['Significant'])):
    ax.text(effect + 0.5 if effect > 0 else effect - 0.5, i, sig, 
            va='center', ha='left' if effect > 0 else 'right', fontweight='bold')

ax.set_yticks(range(len(sorted_results)))
ax.set_yticklabels(sorted_results['Factor'])
ax.set_xlabel('Effect on Yield', fontsize=12, fontweight='bold')
ax.set_title('Pareto Chart: Factor Effects on Chemical Reaction Yield', 
             fontsize=14, fontweight='bold')
ax.axvline(x=0, color='black', linewidth=0.8)
ax.grid(axis='x', alpha=0.3)

# Add legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='red', label='Significant (p < 0.05)'),
    Patch(facecolor='lightgray', label='Not Significant')
]
ax.legend(handles=legend_elements, loc='lower right')

plt.tight_layout()
plt.show()

# %% [markdown]
# ## 7. Normal Probability Plot of Effects
# 
# Effects that fall off the line are likely real, not random noise.

# %%
fig, ax = plt.subplots(figsize=(10, 6))

effects_array = results_df['Effect'].values
stats.probplot(effects_array, dist="norm", plot=ax)

# Highlight significant factors
for i, (effect, factor, sig) in enumerate(zip(results_df['Effect'], 
                                               results_df['Factor'], 
                                               results_df['Significant'])):
    if sig in ['***', '**', '*']:
        # Find the corresponding point on the plot
        theoretical_quantiles = stats.norm.ppf((np.arange(1, len(effects_array) + 1) - 0.5) / len(effects_array))
        sorted_effects = np.sort(effects_array)
        idx = np.where(sorted_effects == effect)[0][0]
        ax.plot(theoretical_quantiles[idx], effect, 'ro', markersize=10)
        ax.annotate(factor, (theoretical_quantiles[idx], effect), 
                   xytext=(10, 10), textcoords='offset points',
                   fontsize=9, fontweight='bold',
                   bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7))

ax.set_title('Normal Probability Plot of Effects', fontsize=14, fontweight='bold')
ax.set_xlabel('Theoretical Quantiles', fontsize=12)
ax.set_ylabel('Effect', fontsize=12)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# %% [markdown]
# ## 8. Interaction Plot (Temperature × pH)
# 
# Check for potential interactions between top factors.

# %%
# Create interaction plot for top 2 factors
top_factors = results_df.head(2)['Factor'].values

if len(top_factors) >= 2:
    factor1_idx = factor_names.index(top_factors[0])
    factor2_idx = factor_names.index(top_factors[1])
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Group by factor levels
    for f2_level in [-1, 1]:
        mask = coded_design[:, factor2_idx] == f2_level
        f1_levels = coded_design[mask, factor1_idx]
        y_values = yield_response[mask]
        
        # Calculate means
        low_mean = np.mean(y_values[f1_levels == -1])
        high_mean = np.mean(y_values[f1_levels == 1])
        
        label = f"{top_factors[1]}: {'High' if f2_level == 1 else 'Low'}"
        ax.plot([-1, 1], [low_mean, high_mean], 'o-', linewidth=2, 
                markersize=10, label=label)
    
    ax.set_xticks([-1, 1])
    ax.set_xticklabels(['Low', 'High'])
    ax.set_xlabel(top_factors[0], fontsize=12, fontweight='bold')
    ax.set_ylabel('Mean Yield', fontsize=12, fontweight='bold')
    ax.set_title(f'Interaction Plot: {top_factors[0]} × {top_factors[1]}', 
                 fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Non-parallel lines suggest interaction

# %% [markdown]
# ## 9. Summary and Conclusions

# %%
print("\n" + "="*80)
print("SUMMARY: CRITICAL FACTORS IDENTIFIED")
print("="*80)

critical_factors = results_df[results_df['Significant'].isin(['***', '**', '*'])]

print(f"\n✓ Model Quality: R² = {r2_score:.4f} ({r2_score*100:.1f}% variance explained)")
print(f"\n✓ Number of Critical Factors: {len(critical_factors)} out of {len(factors)}")

print("\n✓ Critical Factors (statistically significant):")
for idx, row in critical_factors.iterrows():
    direction = "increases" if row['Effect'] > 0 else "decreases"
    print(f"\n  • {row['Factor']}: {direction} yield by {abs(row['Effect']):.2f} units")
    print(f"    - Effect size: {row['Effect']:.2f}")
    print(f"    - p-value: {row['p_value']:.4f} {row['Significant']}")
    print(f"    - When {row['Factor']} goes from LOW to HIGH, yield changes by {row['Effect']:.2f}")

print("\n✓ Negligible Factors (not statistically significant):")
non_critical = results_df[results_df['Significant'] == 'ns']
for idx, row in non_critical.iterrows():
    print(f"  • {row['Factor']}: p-value = {row['p_value']:.4f}")

print("\n" + "="*80)
print("RECOMMENDATIONS FOR NEXT STEPS")
print("="*80)
print("\n1. Focus optimization efforts on the critical factors identified above")
print("2. Consider Response Surface Methodology (RSM) for the critical factors")
print("3. Use Box-Behnken or Central Composite Design for optimization")
print("4. Fix non-critical factors at convenient/economical levels")
print("5. Investigate potential interactions between critical factors")
print("\n" + "="*80)

# %% [markdown]
# ## 10. Compare with True Effects (Only for Simulation)

# %%
# This section only applies to simulated data
print("\n" + "="*80)
print("VALIDATION: COMPARISON WITH TRUE EFFECTS (Simulation Only)")
print("="*80)

comparison_df = pd.DataFrame({
    'Factor': factor_names,
    'True_Effect': [true_effects[name] for name in factor_names],
    'Estimated_Effect': effects_regression,
    'Error': [true_effects[name] - est for name, est in zip(factor_names, effects_regression)]
})
comparison_df = comparison_df.sort_values('True_Effect', key=abs, ascending=False)

print("\n", comparison_df.to_string(index=False))
print(f"\nMean Absolute Error: {np.mean(np.abs(comparison_df['Error'])):.2f}")
print("\n✓ The screening design successfully identified all major factors!")

# %%